In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import pandas as pd

from rate_severity_of_toxic_comments.utilities import *
from rate_severity_of_toxic_comments.model import *
from rate_severity_of_toxic_comments.dataset import *
from rate_severity_of_toxic_comments.training import * 
from rate_severity_of_toxic_comments.preprocessing import *
from rate_severity_of_toxic_comments.tokenizer import * 
from rate_severity_of_toxic_comments.embedding import * 

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
DEFAULT_CONFIG_FILE_PATH = "config/default.json"
LOCAL_CONFIG_FILE_PATH = "config/local.json"

default = open(DEFAULT_CONFIG_FILE_PATH)
CONFIG = json.load(default)

if os.path.exists(LOCAL_CONFIG_FILE_PATH):
    with open(LOCAL_CONFIG_FILE_PATH) as local:
        CONFIG.update(json.load(local))

validate_config(CONFIG)

run_mode = CONFIG["options"]["run_mode"]
df = load_dataframe(run_mode, CONFIG["training"], CONFIG[run_mode])

support_bag = process_config(df, CONFIG)

In [ ]:
df_train, df_valid = split_dataset(df, CONFIG['options']['seed'])

In [ ]:
training_data = build_dataset(df_train, CONFIG["training"]["dataset"], CONFIG[run_mode], support_bag["tokenizer"])
val_data = build_dataset(df_valid, CONFIG["training"]["dataset"], CONFIG[run_mode], support_bag["tokenizer"])

In [ ]:
model, loss_history = run_training(run_mode, training_data, val_data, CONFIG["training"], CONFIG[run_mode], support_bag, CONFIG["options"]["seed"], CONFIG["options"]["wandb"], CONFIG["options"]["use_gpu"], verbose=True, log_interval=100)

In [ ]:
# TODO Evaluation and plotting